<a href="https://colab.research.google.com/github/sarthak247/100daysofML/blob/master/01_Getting_and_Procesing_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount Google Drive

In [0]:
from google.colab import drive
drive.mount('content/drive')

### Downloading Reddit Comments Dataset for January 2015

In [0]:
!wget https://files.pushshift.io/reddit/comments/RC_2015-01.bz2

### Unzipping Dataset and Removing Original File

In [0]:
# Uncompressing Dataset
# This will take around 15-20 mins depending upon the size of dataset chosen
!bzip2 -d RC_2015-01.bz2

### Processing and Filtering out Data from the Dataset

In [0]:
# Importing Required Libraries
import sqlite3
import json
from datetime import datetime
import re

In [0]:
# Connecting to SQL Database
sql_transaction = []
db = sqlite3.connect('RC_2015-01.db')
c = db.cursor()

In [0]:
# Create table in DB
def create_table():
    c.execute("CREATE TABLE IF NOT EXISTS parent_reply(parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)")

In [0]:
# Removing unwanted characters from data
def clean_text(text):
  text = text.lower()
  text = re.sub(r"i'm","i am", text)
  text = re.sub(r"he's","he is", text)
  text = re.sub(r"she's","she is", text)
  text = re.sub(r"that's","that is", text)
  text = re.sub(r"what's","what is", text)
  text = re.sub(r"where's","where is", text)
  text = re.sub(r"\'ll", " will", text)
  text = re.sub(r"\'ve", " have", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'d", " would", text)
  text = re.sub(r"won't", "will not", text)
  text = re.sub(r"can't", "cannot", text)
  text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]", "", text)
  return text

In [0]:
# Batch together a bunch of SQL statements
def transaction_bldr(sql):
  global sql_transaction
  sql_transaction.append(sql)
  
  # Execute if batch exceeds 1000
  if len(sql_transaction) > 1000:
    c.execute('BEGIN TRANSACTION')
    for s in sql_transaction:
      try:
        c.execute(s)
      except:
        pass
    db.commit()
    sql_transaction = []

In [0]:
# Replace the existing comment from the parent-comment pair if a new comment with a higher score shows up
def sql_insert_replace_comment(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?, unix = ?, score = ? WHERE parent_id =?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print('Insert Error: ',str(e))

In [0]:
# Insert a parent-comment pair to the DB
def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('Insert Error: ',str(e))

In [0]:
# Insert only comment to the DB (ie, without a comment associated with it)
def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('Insert Error: ',str(e))

In [0]:
# Check if a parent/comment is of acceptable length
def acceptable(data):
  if len(data.split(' ')) > 1000 or len(data) < 1:
    return False
  elif len(data) > 32000:
    return False
  elif data == '[deleted]':
    return False
  elif data == '[removed]':
    return False
  else:
    return True

In [0]:
# Find a comment by it's comment_id
def find_parent(pid):
    try:
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        return False

In [0]:
# Find existing score of a comment/reply pair
def find_existing_score(pid):
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        return False

### Main Program

In [0]:
create_table()
start_row = 0
cleanup = 3000000 # Cleanup after every 3 million rows
row_counter = 0
paired_rows = 0

with open('RC_2015-01',buffering = 1000) as dataset:
  for row in dataset:
    row_counter += 1
    
    if row_counter > start_row:
      try:
        row = json.loads(row)
        parent_id = row['parent_id'].split('_')[1]
        body = clean_text(row['body'])
        created_utc = row['created_utc']
        score = row['score']
        subreddit = row['subreddit']
        comment_id = row['id']
        
        parent_data = find_parent(parent_id)
        
        existing_reply_score = find_existing_score(parent_id)
        if existing_reply_score:
          if score > existing_reply_score:
            if acceptable(body):
              sql_insert_replace_comment(parent_id,comment_id,parent_data,body,subreddit,created_utc,score)
        else:
          if acceptable(body):
            if parent_data:
              if score >= 15:
                sql_insert_has_parent(parent_id,comment_id,parent_data,body,subreddit,created_utc,score)
                paired_rows += 1
            else:
              sql_insert_no_parent(parent_id,comment_id,body,subreddit,created_utc,score)
      except Exception as e:
        print('Error: ' + str(e))
    
    #Log
    if row_counter % 100000 == 0:
      print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_counter,paired_rows,str(datetime.now())))

    #Clean Up
    if row_counter > start_row:
      if row_counter % cleanup == 0:
        print('Cleaning Up!')
        sql = "DELETE FROM reddit_comments WHERE PARENT IS NULL"
        c.execute(sql)
        db.commit()
        c.execute("VACUUM")
        db.commit()

### Copy Database to Google Drive

In [0]:
!cp RC_2015-01.db /content/drive/My\ Drive